In [1]:
from sklearn.neural_network import MLPClassifier as mlp
import sys
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML")
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML/FeH")
from fitsParser.fitsParser import fitsParser
import pandas as pd
import numpy as np
import re
from numpy import linspace
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from modelValidator import psMetric, cut, shuffleDwarfs, plotRes, plot, norm_inputs, monte_carlo, printTable, conf_matrix
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from function import save_data
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Nadam
binsize = 0.5

def getMags(colour):
    return colour[:colour.index('-')], colour[colour.index('-')+1:]
sdss = fitsParser(includefile = 'FeHdata.txt')
youc = sdss.getAllObj()[1]
cut(youc, metThresh = -5)
youc.data = youc.data[youc.data['FeHadop']<-2.0]
youc.makeColours(N=10)
colours = ['u-g', 'g-i','g-r', 'u-i', 'u-r']
conversion = {
     'u_0' : 'u',
    'CaHK_0' : 'CaHK' 
}


conversion.update({x+'_fpsf' : x for x in 'grizy'})
for col in youc.colours:
    mag1, mag2 = getMags(col)
    youc.data[conversion[mag1]+'-'+conversion[mag2]] = youc.data[col]

for field in youc.data.keys():
    if field in conversion:
        youc.data[conversion[field]]= youc.data[field]
youc.data = youc.data.sample(frac=1)

youc.data = youc.data[youc.data['FeHadop']<-2]
youc.data['test_flag'] = [0]*int(0.8*len(youc.data))+[1]*(len(youc.data)-int(0.8*len(youc.data)))

classes = [-2 - binsize*i for i in range(int((3-2)/binsize)+1)]     

putbin = lambda data : [x + min([c-x for c in classes if c>x]) if x<-2 else -2 for x in data]
youc.data['FeHround'] = putbin(youc.data['FeHadop'])

def baseModel(inputsDim, outDim, Nlayers=30, Nodes = 128):
    model = Sequential()
    model.add(Dense(Nodes, input_dim = inputsDim, activation = 'relu'))
    
    for i in range(Nlayers-1):
        model.add(Dense(Nodes, activation = 'relu'))
        model.add(Dropout(0.2))
    model.add(Dense(outDim, activation = 'softmax'))
    op = Nadam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy'])
    return model

def monte(din, N=3):
    tous = pd.DataFrame(din)
    uncert = pd.DataFrame()
    for col in youc.colours:
        c1 = 'd'+re.compile('.+(?=-)').findall(col)[0]
        c2 = 'd' + re.compile('(?<=-).+').findall(col)[0]
        uncert[col] = np.sqrt(tous[c1]**2+tous[c2]**2)

    uncert[map(lambda x : x[1:], youc.output_uncert)] = tous[youc.output_uncert]
    return monte_carlo(tous[list(youc.colours+youc.outputs)],uncert ,N=N)

montestyle = 2


maxlen = len(youc.data[youc.data['FeHround']==classes[0]])
minlen = min([len(youc.data[youc.data['FeHround']==c]) for c in classes])



if montestyle == 1:
    dprelim = youc.data[(youc.data['FeHround']==classes[0])&(youc.data['test_flag']==0)]
    train = dprelim
    for c in classes[1:]:
        datac = pd.DataFrame(youc.data[(youc.data['FeHround']==c)&(youc.data['test_flag']==0)])
        l = len(datac)
        print 'Num stars in class %.1f: %d'%(c,l)
        if l!=0:
            datac = monte(datac, N = maxlen/len(datac)-1)
            train = train[datac.keys()]
            train = train.append(datac, sort=True).reset_index(drop=True)
if montestyle == 2:
    nmonte = 60
    dprelim  = youc.data[youc.data['test_flag']==0]
    dprelim = monte(dprelim, N = nmonte )
    train = pd.DataFrame()
    dprelim['FeHround'] = putbin(dprelim['FeHadop'])
    for c in classes:
        toapp =  dprelim[dprelim['FeHround']==c]
        train = train.append(toapp.sample(frac = nmonte*1.0*minlen/len(toapp) if nmonte*1.0*minlen/len(toapp)<1 else 1))


train['FeHround'] = [-3 if x<=-3 else -2.5 if x<=-2.5 else -2 for x in train['FeHadop']]
train = train.sample(frac = 1).reset_index(drop= True)
for c in classes:
        datac = pd.DataFrame(train[(train['FeHround']==c)])
        l = len(datac)
        print 'Num stars in class %.1f: %d'%(c,l)

Using TensorFlow backend.



Data saved

condition on dg_fpsf cut 1 inputs
condition on dr_fpsf cut 0 inputs
condition on di_fpsf cut 0 inputs
condition on dz_fpsf cut 0 inputs
condition on dy_fpsf cut 0 inputs
condition on dCaHK_0 cut 0 inputs
condition on du_0 cut 0 inputs
condition on dFeHadop cut 17 inputs
3.13% of inputs cut
Num stars in class -2.0: 2760
Num stars in class -2.5: 2760
Num stars in class -3.0: 2285


In [2]:
class transformer_man:
    
    def __init__(self, classes, func, inverse_func):
        self.classes = classes
        self.func = func
        self.inverse_func = inverse_func

    def transform(self,data):
        return [[1 if self.func(x, c) else 0 for c in self.classes] for x in data]
    
    def inverse_transform(self, trans_data):
        return [self.inverse_func(x) for x in trans_data]
def savemodel(model, params, data, filename):
    joblib.dump(model, filename+'.P')
    joblib.dump(params,  filename+"Params.P")
    joblib.dump(data, filename + "Data.P")

In [4]:
transformer = transformer_man([-2,-2.5,-3],
    lambda x, c :  x == c, 
    lambda x : -2-binsize*x.index(1))

youc.data['label'] = transformer.transform(youc.data['FeHround'])
train['label'] = transformer.transform(train['FeHround'])
for col in youc.colours:
    mag1, mag2 = getMags(col)
    train[conversion[mag1]+'-'+conversion[mag2]] = train[col]

model = baseModel(len(colours), 3, Nlayers = 6,Nodes = 256)
params = {
    'x' : train[colours].values,
    'y' : np.array([x for x in train['label'].values]),
    'verbose' : 2,
    'batch_size': 1000,
    'epochs' : 200,
    'validation_data' : 
    (youc.data[youc.data['test_flag']==1][colours].values, np.array([x for x in youc.data[youc.data['test_flag']==1]['label'].values]))
}

model.fit(**params)


Train on 7805 samples, validate on 132 samples
Epoch 1/200
 - 2s - loss: 1.0913 - acc: 0.3558 - val_loss: 1.0262 - val_acc: 0.6212
Epoch 2/200
 - 1s - loss: 1.0893 - acc: 0.3507 - val_loss: 1.0293 - val_acc: 0.6439
Epoch 3/200
 - 1s - loss: 1.0856 - acc: 0.3707 - val_loss: 0.9699 - val_acc: 0.6212
Epoch 4/200
 - 0s - loss: 1.0826 - acc: 0.3687 - val_loss: 1.0623 - val_acc: 0.4015
Epoch 5/200
 - 0s - loss: 1.0795 - acc: 0.3909 - val_loss: 0.9891 - val_acc: 0.5152
Epoch 6/200
 - 0s - loss: 1.0857 - acc: 0.3748 - val_loss: 1.1064 - val_acc: 0.3485
Epoch 7/200
 - 0s - loss: 1.0772 - acc: 0.3812 - val_loss: 1.0092 - val_acc: 0.4924
Epoch 8/200
 - 1s - loss: 1.0729 - acc: 0.3996 - val_loss: 1.0436 - val_acc: 0.4621
Epoch 9/200
 - 1s - loss: 1.0655 - acc: 0.4202 - val_loss: 0.9593 - val_acc: 0.4773
Epoch 10/200
 - 0s - loss: 1.0970 - acc: 0.3791 - val_loss: 1.0737 - val_acc: 0.4091
Epoch 11/200
 - 0s - loss: 1.0822 - acc: 0.3967 - val_loss: 1.0483 - val_acc: 0.4545
Epoch 12/200
 - 0s - loss: 

Epoch 97/200
 - 0s - loss: 1.0180 - acc: 0.4676 - val_loss: 0.8793 - val_acc: 0.6591
Epoch 98/200
 - 0s - loss: 1.0261 - acc: 0.4564 - val_loss: 0.9635 - val_acc: 0.5076
Epoch 99/200
 - 0s - loss: 1.0202 - acc: 0.4721 - val_loss: 0.8294 - val_acc: 0.6667
Epoch 100/200
 - 0s - loss: 1.0290 - acc: 0.4502 - val_loss: 1.0720 - val_acc: 0.4242
Epoch 101/200
 - 0s - loss: 1.0129 - acc: 0.4732 - val_loss: 0.9315 - val_acc: 0.5379
Epoch 102/200
 - 0s - loss: 1.0180 - acc: 0.4666 - val_loss: 1.0367 - val_acc: 0.4242
Epoch 103/200
 - 0s - loss: 1.0220 - acc: 0.4582 - val_loss: 0.8699 - val_acc: 0.6212
Epoch 104/200
 - 0s - loss: 1.0238 - acc: 0.4620 - val_loss: 1.0568 - val_acc: 0.3788
Epoch 105/200
 - 0s - loss: 1.0352 - acc: 0.4439 - val_loss: 0.9142 - val_acc: 0.6515
Epoch 106/200
 - 0s - loss: 1.0222 - acc: 0.4673 - val_loss: 1.0982 - val_acc: 0.3864
Epoch 107/200
 - 0s - loss: 1.0194 - acc: 0.4627 - val_loss: 0.9772 - val_acc: 0.5000
Epoch 108/200
 - 0s - loss: 1.0157 - acc: 0.4719 - val_lo

Epoch 193/200
 - 0s - loss: 0.9821 - acc: 0.4843 - val_loss: 0.9089 - val_acc: 0.5606
Epoch 194/200
 - 0s - loss: 0.9814 - acc: 0.4993 - val_loss: 0.8016 - val_acc: 0.6667
Epoch 195/200
 - 0s - loss: 0.9946 - acc: 0.4865 - val_loss: 0.8693 - val_acc: 0.5758
Epoch 196/200
 - 0s - loss: 0.9894 - acc: 0.4871 - val_loss: 0.9005 - val_acc: 0.5909
Epoch 197/200
 - 0s - loss: 0.9894 - acc: 0.4906 - val_loss: 0.8999 - val_acc: 0.5606
Epoch 198/200
 - 0s - loss: 0.9954 - acc: 0.4821 - val_loss: 0.9055 - val_acc: 0.5606
Epoch 199/200
 - 0s - loss: 0.9897 - acc: 0.4906 - val_loss: 0.9410 - val_acc: 0.5076
Epoch 200/200
 - 0s - loss: 0.9871 - acc: 0.4910 - val_loss: 0.9107 - val_acc: 0.5758


In [5]:
pretrans = lambda data : [[1 if False not in [x[i] > y or y == x[i] for y in x ] else 0 for i in range(len(x))] for x in data] 
pretrans2 = lambda data : [[0,0,1] if x[2]>0.7 else [0,1,0] if x[1]>0.33 else [1,0,0] for x in data]
z = model.predict(youc.data[colours])
youc.data['FeH_pred'] = transformer.inverse_transform(pretrans(z))
            
youc.data['FeH'] = youc.data['FeHadop']
youc.data['FeHadop'] = youc.data['FeHround']
youc.data['FeHround_prist'] = putbin(youc.data['FeH_pristine'])
i = list(classes)


data = youc.data
pur, com = psMetric(data, inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(data, classes), columns = classes, index = classes) , '\n'      
             
pur2, com2 = psMetric(data[data['test_flag']==1], inc = i)
print printTable(i,pur2,com2)
print pd.DataFrame(conf_matrix(data[data['test_flag']==1], classes), columns = classes, index = classes),'\n'  


pur3, com3 = psMetric(data, inc = i, pred = 'FeHround_prist')
print printTable(i,pur3,com3)
print pd.DataFrame(conf_matrix(data, classes, pred = 'FeHround_prist'), columns = classes, index = classes),'\n'  



| [Fe/H]< | purity | completness |
----------------------------------
| -2.0    | 1.0000 | 1.0000      |
| -2.5    | 0.5069 | 0.7273      |
| -3.0    | 0.1420 | 0.5000      |

          -2.0      -2.5      -3.0
-2.0  0.555831  0.250620  0.193548
-2.5  0.285024  0.420290  0.294686
-3.0  0.217391  0.282609  0.500000 

| [Fe/H]< | purity | completness |
----------------------------------
| -2.0    | 1.0000 | 1.0000      |
| -2.5    | 0.5538 | 0.7200      |
| -3.0    | 0.1212 | 0.5714      |

          -2.0      -2.5      -3.0
-2.0  0.646341  0.134146  0.219512
-2.5  0.302326  0.441860  0.255814
-3.0  0.142857  0.285714  0.571429 

| [Fe/H]< | purity | completness |
----------------------------------
| -2.0    | 1.0000 | 1.0000      |
| -2.5    | 0.4081 | 0.9921      |
| -3.0    | 0.1098 | 0.6304      |

          -2.0      -2.5      -3.0
-2.0  0.096774  0.531017  0.372208
-2.5  0.009662  0.579710  0.410628
-3.0  0.000000  0.369565  0.630435 



In [ ]:
for c in classes:    
    print len(train[train['FeHround']==c]), c

In [ ]:
def save(model, colours, classes, binsize, data, filename):
    try:
        joblib
    except NameError:
        import joblib
    
    joblib.dump(model, filename+".P")
    joblib.dump(
    {
        'colours' : colours,
        'classes' : classes,
        'binsize' : binsize
    }, filename+'Params.P'
    )
    joblib.dump(data, filename+'Data.P')

In [ ]:
#save(model, colours, classes, binsize, youc.data, 'april16v3')
print sum([ 1 if x else 0 for x in youc.data['FeHadop']==-3])


In [ ]:
train['FeH_pred'] = [classes[x.index(1)] for x in pretrans(model.predict(train[colours]))]
pur, com = psMetric(train, inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(train, classes), columns = classes, index = classes) , '\n'  


pur, com = psMetric(data[data['test_flag']==0], inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(data[data['test_flag']==0], classes), columns = classes, index = classes) , '\n'  